# Definizione delle funzini

In [ ]:
import requests
import matplotlib.pyplot as plt
import numpy as np
import lxml.html
from lxml import etree
import xml.etree.ElementTree as ET
import string
import csv
import numpy as np
import time

def AbsolutePaths(tags):
    tmp=''
    for tag in tags:
        tmp+=tag.tag+'/'
    return str(tmp)    
  
def xpath_generator_gz(C): #C è un dizionario
  X = []
  for link in C:
    #Analizza l'elemento html della pagina
    tree = lxml.html.fromstring(C[link])
    #ritorna la lista dei nodi foglia
    childs = tree.xpath('//*[not(child::*)]')
    #funzione che calcola il path assoluto radice-foglia come stringa
    #calcolo della lista che contiene gli xpath assoluti alle foglie della pagina 
    for child in childs:
      #ciascun elemento in X è una quadrupla (implementata come una lista di 4 elementi) formata da:
      #url della pagina, xpath di ciascun nodo della pagina, contenuto del nodo, peso del nodo
      xpath_element = []
      tags = child.xpath('ancestor::node()')
      path = AbsolutePaths(tags)
      path += str(child.tag)
      contenuto = child.text_content()
      xpath_element.append(link)
      xpath_element.append(path)
      xpath_element.append(contenuto)
      xpath_element.append(0)
      X.append(xpath_element)
  return X

def xpath_generator(C): #C è una lista
  X = []
  for link in C:
    #ritorna la get
    link = ''.join(link)
    link = link[:len(link)-1]    
    response = requests.get(link)
    #ritorna l'elemento html della pagina
    tree = lxml.html.fromstring(response.text)
    #ritorna la lista dei nodi foglia
    childs = tree.xpath('//*[not(child::*)]')
    #funzione che calcola il path assoluto radice-foglia come stringa
    #calcolo della lista che contiene gli xpath assoluti alle foglie della pagina 
    for child in childs:
      #ciascun elemento in X è una quadrupla (implementata come una lista di 4 elementi) formata da:
      #url della pagina, xpath di ciascun nodo della pagina, contenuto del nodo, peso del nodo
      xpath_element = []
      tags = child.xpath('ancestor::node()')
      path = AbsolutePaths(tags)
      path += str(child.tag)
      contenuto = child.text_content()
      xpath_element.append(link)
      xpath_element.append(path)
      xpath_element.append(contenuto)
      xpath_element.append(0)
      X.append(xpath_element)
  return X

#Ricerca la frequenza e la frequenza del contenuto di un certo xpath
#e la cardinalità dell'insieme di tutti i contenuti associati a quell'xpath
def find_f(url, xpath, contenuto, lista):
  frequenza = 1
  f_contenuto = 1
  #t_i è l'insieme di tutti i contenuti associati a xpath
  t_i = [contenuto]
  #lista dei url già visti nella ricerca
  url_visti = []
  url_visti.append(url)
  for new_path in lista:
    #se allo stesso xpath ho un contenuto diverso da quelli che già ho lo inserisco in t_i
    if new_path[1] == xpath and new_path[2] not in t_i:
      t_i.append(new_path[2])
    #calcolo della frequenza e di f_contenuto
    if new_path[0] not in url_visti:
      if new_path[1] == xpath:
        url_visti.append(new_path[0])
        frequenza += 1
        if new_path[2] == contenuto:
          f_contenuto += 1
  return frequenza, f_contenuto, len(t_i)

#lista = [(url,xpath,contenuto,peso), (....), (....), .... ]
def calcolo_pesi_xpath(lista, num_pagine):
  for xpath in lista:
    #prendo xpath
    x = xpath[1]
    url = xpath[0]
    contenuto = xpath[2]
    #calcolo la frequenza, la frequenza del contenuto e la cardinalità di t_i
    frequenza, f_contenuto, t_i = find_f(url, x, contenuto, lista)
    #calcolo della informatività
    info = 1 - (f_contenuto / (num_pagine * t_i))
    #calcolo del peso del xpath
    peso = frequenza * info
    #inserisco il peso nella lista
    xpath[3] = peso
    
def somma_pesi_per_pagina(X):
  url_pesi = {}
  for e in X:
    k = e[0]
    if k not in url_pesi.keys():
      url_pesi[k] = e[3]
    else:
      url_pesi[k] += e[3]
  return url_pesi

#Trova la pagina (ritornando il suo url) che ha massima la somma dei pesi di tutti i suoi xpath
def find_p(X):
  #creo un dizionario che ha come chiave l'url di una pagina e come valore la somma di tutti i pesi dei suoi xpath
  pagine_con_pesi = somma_pesi_per_pagina(X)
  peso_max = 0
  pagina_max = ''
  #ricerco la pagina che ha valore massimo nel dizionario creato precedentemente
  for k in pagine_con_pesi.keys():
    peso_pagina = pagine_con_pesi[k]
#     print(k, peso_pagina)
    if peso_pagina > peso_max:
      pagina_max = k
      peso_max = peso_pagina
#   print('\n')
  #ritorno l'url della pagina trovata
  return pagina_max, peso_max

#Crea un dizionario con chiave xpath e valore il contenuto
def build_dict(X, p):
  d = {}
  for e in X:
    if e[0] == p:
      if e[1] not in d:
        xpath = e[1]
        content = e[2]
        d[xpath] = content
  return d

#Rimuove la pagina da X e gli xpath associati ad essa
def remove(X, p):
  #nuova lista da riportare
  X1 = []
  dict_xpath_content = build_dict(X,p)
  for e in X:
    #se l'xpath è contenuto in una pagina diversa da p
    #e ha un xpath non presente in p allora lo aggiungo a X1
    if e[0] != p and e[1] not in dict_xpath_content:
      X1.append(e)
  return X1

# ============= MAIN =============
def page_annotation(C, K):
  #X = xpath_generator(C)
  X = xpath_generator_gz(C)
  print('XPath totali: ' + str(len(X)))
  calcolo_pesi_xpath(X, len(C))
  S = []
  pesi = []
  while X != [] and len(S) <= K:      
    p, peso = find_p(X)
    print(p)
    S.append(p)
    pesi.append(peso)
    X = remove(X, p)
  return S, X, pesi

# Carico il dataset
Il dataset preso in considerazione è quello di 'bestbuy'

In [ ]:
import gzip
#contiene come chiave il nome del file e come valore la pagina html
C={}
i = 1
#prendo le prime i pagine
while i < 50:
  file = str(i) + '.html.gz'
  #il link di apertura può variare in base a cosa stiamo analizzando
  with gzip.open('/dataset/www.bestbuy.com/pages/'+ file, 'rb') as f:
    file_content = f.read() #html
    C[file] = file_content
    i+=1

# Start

In [ ]:
S, x, pesi = page_annotation(C,5)

print(len(x)) #stampiamo il numero di xpath rimanenti

# Prove per le metriche dell'algoritmo
precizion e recall